<a href="https://colab.research.google.com/github/sarahalyahya/SoftwareArt-Text/blob/main/FinalProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#The Unsent Project | Sent(?) 
###Idea
The idea from my final project was inspired from our assignment 2 and the artist talk with Lauren Lee Mccarthy. Throughout this course, I've been interested in exploring human emotions and interactions which is why I kept finding myself going back to the "The Unsent Project". With my final project, I wanted to experiment with moving things outside the computer screen, discomfort, and human conversations. So, I decided to train GPT-2 to generate texts like the ones on the Unsent Project and then create a simple conversation "Toolbox" which can allow me to pick random machine-generated responses to use in a conversation based on the sentiment of the received message. 
###Steps
####Scraping
For this part, I used BeautifulSoup to scrape through 10 thousand texts from The Unsent Project and followed steps similar to what I did in the previous assignment. I went through each text, used TextBlob to make sure it was in English and translate it if it isn't. Finally, I ran a sentiment analysis on it to avoid any texts with polarity = 0. The final output from this step is a text document that I used to train GPT-2. 
####GPT-2
This time, I had much more tokens to train the GPT-2 which helped in getting results that sounded more realistic and coherent. However, I wanted to maintain some of the randomness as I thought it could result in interesting reactions from people. 
####Processing Outputs
I put all the GPT-2 outputs into a string, split it into a list and cleaned it up using Python string operations. Then, I created two separate arrays of positive generated texts and negative generated text for later usage.
###The Cards 
I printed out some texts to create prompt cards. Ideally, I wanted to throw these around campus, but given the current health guidelines, plus the fact that my phone number was on the cards, I chose to experiment with having friends give them to their friends and see what happens.
On one side, the card had the unsent text, on the other it "You Just Received this Text on your phone. How would you respond? Whatsapp your reply to:"
###The Conversations Toolbox
From there on, it was a waiting game to see when someone will finally text me. For the conversations, I created a "conversation toolbox," that takes the received text as input and asks if I have any prompt for the answer. If I don't have a prompt, it gives me a previously generated text based on the sentiment of the received text. However, if I add a prompt, it runs the GPT-2 again, uses the prompt as the prefix, and generates a new text.
###Video
I chose to present my work in video format. I felt like it was the closest in communicating the experience to the viewers, something like an experimental film. 
###Reflection
The main tools I used from our course were:

1.   BeautifulSoup
2.   GPT-2
3.   TextBlob (For polarity and translation)

I tried to put these tools together in a way that can help me create an impactful real-life experience for the user. They were generally simple algorithms, but most of the weight was placed on the output and the human interaction in this project. I was mainly working towards getting reactions from people and interacting with them in an unfamiliar way. This project revealed a lot about how people try to adapt to artifical intelligence, even when they find it confusing. In many cases, it felt like they overlooked all the GPT-2 glitched that I hoped would get me a confused reaction from them. 
I think this project can still grow a bit more. I would be interested to see actual strangers interacting with this human/bot mix that I created, and see what a refined more large-scale version would yield. 

# [The Video](https://https://youtu.be/WqXlAwy2a3w)

#Some Reflections
##Stranger 1
"Overall I’d say this was a fairly strange experience. Even form the first response I received, I was slightly confused. Because it said “-m” and then in brackets explained that (this was written before a breakup occurred as I was back to being my normal self,)” so I was just thinking: what? what is going on? What breakup? Also the “back to being my normal self” part I couldn’t understand. I didn’t say I was confused from the get go because I thought I’d get more clarification from the following messages but I just got more and more confused. Then there was the “From Wowpedia The Shining...” message which made me think that the respondent was sending me quotes from the book. I thought for a second that that could’ve been a character in the book. I speculated that I was just speaking to a bot at some points but I still tried to make sense of things somehow without trying to jump to that conclusion too quick. Then, by the end I was pretty confident that these were auto generated messages. So I just gave up with trying to give a solid response because none of the responses I was receiving made much sense."

##Stranger 2 - A snippet from the whatsapp chat after the conversation ended

[05:00] Sarah: how was that experience any reflections?
<br>[05:00] Stranger 3: I love it
<br>[05:01] Sarah: why?
<br>[05:01] Sarah: any specific reason?
<br>[05:01] Stranger 3: i don’t have deep talks with people
<br>[05:01] Stranger 3: a bot did just fine
<br>[05:03] Sarah: u can always come to the bot
<br>[05:03] Stranger 3: i might actually

In [ ]:
import requests
from bs4 import BeautifulSoup
import random
from textblob import TextBlob

In [ ]:
##Part 1 - Scraping and Processing Texts from The Unsent Project
texts = []
f = open("UnsentTexts.txt", "a")
for i in range (10000):
  unsentProjectLink = "https://theunsentproject.com/posts/" + str(800000 + i)
  reqUnsentProject = requests.get(unsentProjectLink)
 
##This if statement skips any broken links
  if reqUnsentProject.status_code == 200:
    textSoup = BeautifulSoup(reqUnsentProject.content,'html.parser')
    element = textSoup.find('img', alt= True)
    text = (element['alt'])
    blob = TextBlob(text)
  #removing the first part, "To: X -", as it will always return english as the language and won't translate the content
    forTranslation = blob.split("-", 1)
    

  #checks if the text is in english, if not, it translates it to english
  #The reasoning behind putting these in Try-Except blocks is because TextBlob occasionally fails at translating a text. This caused my loop to crash a few times.
    try:  
      if (forTranslation[1].detect_language() != 'en'):
        
            forTranslation[1] = forTranslation[1].translate(to='en')
    except:
      i+=1
    #checking for polarity to avoid any 0 polarity texts. The goal was to feed GPT-2 stuff that are clearly positive or clearly negative
    sentAnalysisText= TextBlob(text)
    polarity = sentAnalysisText.sentiment.polarity
    if polarity != 0:
        text = '- '.join(forTranslation)
        ##basic string operations and then adding them all to a text file
        texts.append(text)
        text = text + '\n' + '*******************************************************************************************************************' + '\n'
        f.write(text)
        i += 1

    else:
        i+=1
   
  else:
    i += 1
  
f.close()


In [ ]:
##Part 2 - GPT-2
%tensorflow_version 1.x
!pip install -q gpt-2-simple

In [ ]:
import gpt_2_simple as gpt2
import tensorflow as tf
from datetime import datetime
from google.colab import files

In [ ]:
!nvidia-smi

In [ ]:
!git clone https://github.com/openai/gpt-2.git 2> /dev/null || git -C gpt-2 pull
!python gpt-2/download_model.py 355M

In [ ]:
run_name = "gpt_uploaded"

from google.colab import files

uploaded = files.upload()

file_name = list(uploaded.keys())[0]
print("Received file " + file_name)

Saving UnsentTexts.txt to UnsentTexts (1).txt
Received file UnsentTexts.txt


In [ ]:
tf.reset_default_graph() # Reseting allows us to finetune multiple times
                         # Run this cell again for more finetuning

sess = gpt2.start_tf_sess()

steps = 50 # Number of times to train
sample_every = 3 # Ask the model to generate "original" text as we train so we can
                  # see the progress - the generated text should start to match
                  # our sample text.
sample_length = 25 # How long of sample text to generate

gpt2.finetune(sess,
              dataset=file_name,
              model_name='355M',
              checkpoint_dir = 'checkpoint',
              steps=steps,
              run_name=run_name,
              print_every=1,
              sample_every=sample_every, 
              sample_length=sample_length,
              )

In [ ]:
tf.reset_default_graph() # Reseting allows us to finetune multiple times
                         # Run this cell again for more finetuning

sess = gpt2.start_tf_sess()

steps = 20 # Number of times to train
sample_every = 3 # Ask the model to generate "original" text as we train so we can
                  # see the progress - the generated text should start to match
                  # our sample text.
sample_length = 20 # How long of sample text to generate

gpt2.finetune(sess,
              dataset=file_name,
              model_name='355M',
              checkpoint_dir = 'checkpoint',
              steps=steps,
              run_name=run_name,
              print_every=1,
              sample_every=sample_every, 
              sample_length=sample_length,
              )

In [ ]:
tf.reset_default_graph() # Reseting allows us to finetune multiple times
                         # Run this cell again for more finetuning

sess = gpt2.start_tf_sess()

steps = 30 # Number of times to train
sample_every = 3 # Ask the model to generate "original" text as we train so we can
                  # see the progress - the generated text should start to match
                  # our sample text.
sample_length = 20 # How long of sample text to generate

gpt2.finetune(sess,
              dataset=file_name,
              model_name='355M',
              checkpoint_dir = 'checkpoint',
              steps=steps,
              run_name=run_name,
              print_every=1,
              sample_every=sample_every, 
              sample_length=sample_length,
              )

In [ ]:
 gpt2.generate(sess,
              run_name=run_name,
              length=30,
              temperature=0.7,
              prefix="",
              nsamples=20,
              batch_size=20
              )



In [ ]:
##Part 3 - Working with the Output 
#I put all my generated outputs into a big string, then cleaned them up and created a general list
output = '''One of the more interesting developments is how the US has changed since I've been here. I think I've become better at understanding each and every one
====================
You are here: Help us Help you - I was in love with you, and of course you didn't love me back, you were worried

====================
Ratner is the first man I've ever loved. There are so many things I wish I could say to him, I hope he finds someone
====================
What are the odds that you will ever fall in love with anyone?

I think that I will, but I know this doesn’t
====================
Trying to do your homework is not easy, but you have to realize that you can't teach someone how to learn. You can't make them
====================
CLEVELAND, OH - GREETINGS TO: KATHY D. -  I wish you'd come back to me. I miss you
====================
I'm sorry. I'm sorry for everything. I'm sorry for not being enough to you. I'm sorry for the things I did. I
====================
I never really thought I would be able to love you like I loved you because of your red hair.
*******************************************************************************************************************
To:
====================
As a woman who was raised by a single mother in a single-parent home, I know that I'm not enough for anyone. It's not
====================
You must sign in or register to continue reading content.

The lights of the home we built are eternally bright, but when I think of
====================
To: bryon -  sometimes i worry i don't fit in with the people you surround yourself with. you treat me like second class,
====================
․ I just want to talk to you. I think we're gonna be great together

․ I really like you. I hope you
====================
’ on the bier

dude. i didn’t know if i wanted to live with you or not.

i
====================
I was always a princess, but I am still trying to be a man.
*******************************************************************************************************************
To: tony -  thank
====================
DUBLIN
Is it just me, or do you just want to have sex with me first? I hate that I'm the only one
====================
You said you loved me and I just didn’t feel the same about you. I wish you could have said you still do. But I
====================
I would love to talk to you but I'm too scared

I don't think I can

I'm afraid of not being able to
====================
YEARLORD - c. 1723
Recent Examples on the Web

The relationship between her and the sun never developed, but the love
====================
Credit: Stanislaus -  I don't think you understand, but I can't leave you without risking my own life.
************************************************************************************************
====================
My first taste of u was so good. I wish I could remember what the first time was like. I wish I could remember what it felt like
====================
I'm so glad I have you. I'm so happy I could help you.
*******************************************************************************************************************
To: james - 
====================
‹ I’m sorry that we didn’t work out. I was too damaged to be broken by a girl I met at the
====================
You're the kind of person that doesn't even know you're gay.
*******************************************************************************************************************
To: jenny -  i think
====================
I had a dream about you. You told me, "I love you. I'm so happy you're happy." I said to myself, "
====================
I have always loved the sound of your voice. You are such a sweet and gentle person.
*******************************************************************************************************************
To: daniel
====================
I can't stop thinking about you.
*******************************************************************************************************************
To: Ali -  I don't think I ever really felt real with
====================
✓
I am so sorry for how I treated you.

You were my first love, but I also did everything in my power to
====================
I'm sorry. I'm really sorry I ruined your life. I'm sorry I made you feel this way.
*******************************************************************************************************************

====================
I love you so much but i hate you so much.
*******************************************************************************************************************
To: jake -  i cant stand you.
====================
I was so jealous of him. I didn’t even know what I wanted.
*******************************************************************************************************************
To: jaden -
====================
You are not a member of our family. You are a friend.
*******************************************************************************************************************
To: C.T. -  I
====================
You had me at "charming". I loved the way you treated me when I was so sick and broken.
*******************************************************************************************************************

====================
‹Why’m you still here?

‹Why’m you still here?

I don’t think
====================
To: danielle -  i just thought we were meant to be, but it was for a little while. i miss you. i miss
====================
Here's what you need to know...

I was 14 when I lost my virginity to you. You never really made up your mind on me
====================
You're the only person i think will.
*******************************************************************************************************************
To: katie -  i love you, i love you
====================
I've been in love with you for a while now but I still don't know what to do with myself
*******************************************************************************************************************
To
====================
Our Story

I was born with dwarfism.

I was raised by my mom and dad and I still have a lot of respect for
====================
You may have heard of me before — you may not know I'm a vegan. But I don't think you'll ever know that I'm a
====================
For the past decade, I've tried to be the person that gets to have my cake and eat it too. I've tried to be the person
====================
For the past 6 months, I have been afraid to communicate to you because I told you I didn't want to be your
====================
It was good that I was happy, it meant I wasn't alone.
*******************************************************************************************************************
To: ben
====================
The Quiet One

As you know, I met my future wife. I met her after we broke up. I have
====================
The first time we met was at our first night together. I said yearning to see you again and I said that I
====================
We do not have a typical final day of the school year.
*******************************************************************************************************************
To: mikey
====================
To: kat -  i miss you so much. i still think about you everyday. i think about you in a
====================
LOOK AT THIS!!! I LOVE THIS!!!!
-TK
*******************************************************************************************************************
To: Felicia -
====================
To: jiampo -  i think you like me more than you think you do. i think we'll
====================
The next generation of our product is coming!

We are so excited to share our new product with you. Feedback will
====================
We can't have any secrets!


-Max

-s

P.S. I'm still
====================
[Guild Profile]
Chandra - the destroyer of worlds

Per: Kielly

Age: 20
====================
You might be able to play with my hair. You might be able to look at me and say, "I know you
====================
What is it with you and me?


I'm sure we both know we're not good enough for each other.
====================
Highlighted:

- The best part is yet to come.

- I hope to see you again
====================
As the world gets smaller and the distance between us grows, I wonder about you.
To: Alex -  I loved
====================
Noah.

You're the most hated person in the world. You're a bully, and a liar.
====================
At the end of the day, I'm still a dog.
And I still can't stop loving you.
But
====================
D.A.
So, you wanna go to college. I know I did. I mean, I'm sure you
====================
I wish I could have been more like you. Because I was meant to be with you forever.
****************************************************************************************************************
====================
You might think about it and think, "I wish I could have had a part of you that I could've been my
====================
‹ I'm sorry, I just wanted to be able to help you.‹
‹ I’m sorry I wasn’t better for you.
====================
Gambler's. I used to have a crush on you, but you broke me.
*******************************************************************************************************************
To: juan b -  i miss
====================
To: ronald -  i think about you all the time. you're a really good guy. i just wish you understood that. i miss you so much.
====================
Let's talk about how rad you are.
*******************************************************************************************************************
To: T. -  I see your blue eyes and I still think about you, but
====================
We're still waiting on the birth certificate, and I don't think I ever really did love you, because I didn't want to hurt you. You're a kind of
====================
By: Aveeno

by Aveeno

Honey. I love you

’m

I love you so much

’m

====================
I can't believe I let you go. I could have made it work.
*******************************************************************************************************************
To: alan -  i can't believe i still
====================
I am so happy that you are my first love and that you have made me so happy. I fell in love with you when I was still in high school and I
====================
This is the first blog post since I left for college. I'm so happy to be back to writing for you. Here's hoping that you find your way back to me
====================
This is the article on the legendary Gunther. You may know him as the head of the Homunculus Club in the anime.
*******************************************************************************************************************
To
====================
To: Mikael -  I never, ever tried to fix your stupid eyes without first asking you what you'd do for something so simple. I hope you never have to
====================
We do have a lot of issues here, I think. We're not perfect, but we probably do a decent job of it.
*******************************************************************************************************************
To
====================
So I wanted to ask you one, but I already did. Why is it that you have never really accepted me? I ask you this because I don’t know
====================
When I was thirteen I was diagnosed as bisexual. I was devastated. I have always been a person who likes to romanticize love and I will forever. I feel like we
====================
Do you have a special place in your heart for me?
-m
(This was written before a breakup occurred and I was back to being my normal self,)

====================
You know you have to, because you are fucked.
*******************************************************************************************************************
To: Kaitlyn -  I wish I could tell you how much I love
====================
The story follows new love and the change that it brings to her. 
The best way I can describe it is this - It's like life without us and I don
====================
We're all in this together, we can't you see that? we all do it
*******************************************************************************************************************
To: s.austin -  i
====================
We were all so young. I was just 11. I was so naive and I really do tear up when I think of the way we treated eachother. I wish we
====================
What do we have that neither of us can see?
- David
*******************************************************************************************************************
To: Kayla -  I miss you so much. But no
====================
We've got tons of free downloads for you!

Mini-Girls

I Love You And I Don’t Have To Be You

I know I
====================
A graduate of the University of Newcastle and a former friend of mine, who I thought I was going to be best friends with since we were in junior high school. I hate
====================
‹I got some good news for you.‹
‹We’ve become good friends and we’ve been best friends for way longer. I

====================
Crazy good! I love that you know how much I love you.
*******************************************************************************************************************
To: jimmy -  i don’t
====================
"I love you so much but you hurt me so much. I wish I didn't. I wish you would care for me and not just for yourself. I don't
====================
EAST LANSING - A couple of weeks ago, I asked you how you felt about your first love. You said you wanted to be with her, but you were
====================
‭ I don’t think I ever really loved you.

I mean, you never did.

But I still think you’re one of
====================
BALTIMORE (WJZ) — One of my best friends is gay. 6'3" and so fat.
*******************************************************************************************************************
To:
====================
The text of the letter was over a year in the making, and it was written in a desperate attempt to save her life. The more I think about it, the more
====================
If the word 'sissy' makes you uncomfortable I will not tell you how I felt.
*******************************************************************************************************************
To: bryon -  i�
====================
So, kind of a fake news story.
*******************************************************************************************************************
To: jacob -  you hurt me so much. my gut feeling was that you
====================
‡
I have never liked you more. I have loved you to the point that I will never tell you

‡
I love you so much and I
====================
We've all seen the photos of our friends and family. They're cute and all, but like all our friends, they get lost in the shuffle.
************************************************************************************************
====================
I only wish I had never left. I loved you so much. I've never loved someone like that before.
*******************************************************************************************************************
To: daniel g
====================
You say you're sorry, but you don't. To you, it doesn't make sense that you would have hurt me the way you did. Even if it was just
====================
Sebastian Hill's mom, who works at McDonald's, said she was mad at him for not caring about her. She told him he could be friends with her
====================
So, I'm like, 'Wow, I miss you. I'm ready to move on,' but I can't do that. I don't want to be a burden
====================
As the years went by, I kept seeing videos of you. I thought you were so cute and I wanted to be a part of you, but I will always care about
====================
Why is it that when you have children, you make them molly?
-J
To: jay -  i always thought i was way too good for you
====================
‹’”‖ I love you. I need to let you go. I know you don’t know yet, but
====================
A man who killed his former girlfriend, then left her for good, is now on the run.
*******************************************************************************************************************
To: tra
====================
People often ask me, "when was the last time you really loved me?"
I have to admit I fall for you. I honestly don't
====================
Manifest
A shadow of a man.
- Noah O.

I have a lot of love for you, I just hate you
====================
Greetings, missy, are you sick or something?
*******************************************************************************************************************
To:Matt -  I miss you, though I
====================
***
"I'm sorry but that was the last time we ever talked. You don't even like me but you invited me over to your place
====================
I don’t know if I can believe that I could ever love you like I loved you.
*******************************************************************************************************************
To:
====================
A few months ago, I was in a bad place and I made a decision I wish I could undo. I let myself be manipulated by a person
====================
 I'm so sorry that I let you go. I couldn’t help myself. I know you don’t like me anymore but I
====================
BUTLER: For the last 20 years we have been best friends. I used to be so jealous of you and all the girls you went to school
====================
The athlete has been with her boyfriend of over 3 years now, who she met at the same time, and is now a good friend.

====================
Jan 21, 2015; Oakland, CA, USA; Oakland Raiders wide receiver Amari Cooper (17) is tackled by San Francisco 49ers defensive tackle
====================
I am no longer a fan of her - i think some of her best behavior was due to her pushing me away and i wish i didn’
====================
If you ever find yourself in a situation where you don’t feel confident enough to talk to someone, just hold back and let them deal with
====================
You will never understand how much you mean to someone.
*******************************************************************************************************************
To: jonathan p. -  i love you
====================
We so love you and your funny face, and I love that you're excited to meet people.
*******************************************************************************************************************
To: j
====================
Hello everyone! I'm glad you're doing well!
-M
I'm not sure you will ever see this but I just wanted to say
====================
I have a confession to make. I really don't like you. I just don't. I wish we could go back to the days when we
====================
To: m -  i think about you most days and i never forget it. i think about you everyday and i always want to be your friend
====================
We have a lot to talk about.

I wish we could just talk about something.

I don’t know what we have
====================
Our first question to you was, "How did you feel when you were in my life? Did I ever love you?"
-G
***
====================
Most of us pride ourselves on being good friends. We laugh and talk and talk and talk. It's not often you meet someone who deserves to be
====================
I recently read one of your books and I love it so much. I'm so happy I did. I have to admit that I was a little
====================
And you think you can handle it.
*******************************************************************************************************************
To: katie -  you made me feel so scared to take
====================
Some people are born with claws on their hands. Some people have claws on their feet.
-- 
I was called a bitch when I didn
====================
Brief summary:
Anime is where I realized I was not really good enough for anyone, even if I tried.
-Aurel
====================
Welcome to @kimberlyn - i really love you. but i'm also going to love you when we get married.
****************************************************************
====================
I've never been good at love. I've never been in love with a girl. I've never been in love with a boy.
********************************
====================
‹‹
This is for the jolly christian. he's an old friend and i miss him, i miss our friendship, i
====================
‹ under the circumstances?’
‹ I’ve never felt this way before, but it’s my first real love
====================
I love you, always.
*******************************************************************************************************************
To: Aiden -  I still think about you sometimes. Do you still
====================
"I'm sorry I ruined your life.
-Will"
–Isaac, 17
After last night, I'm starting to think I
====================
What does summer mean to you?
To me it means that you like to play with your friends and you want to make friends with every girl in
====================
***************************************************************************************************
To: Alyssa -  i love you. i've never loved someone like i love you. i'll always
====================
1. In a similar way, but in a different time. -  I'm sorry that I didn’t stop you from hurting me.
====================
Sometimes I wonder whether we're not meant to be.
*******************************************************************************************************************
To: joshua -  i hate you, i
====================
***************************************************************************************************
To: jay -  you are the best.
*******************************************************************************************************************
To: jay - 
====================
You can use the "D" to express love and the "I" to express what you mean to me. I do not know if I ever
====================
Hey everyone,
I know I'm not the most popular one here. I do apologize for not responding to you sooner, but I still think I
====================
Well, we got to know each other, but, at the same time, I'm worried i'm not enough for you.
************************************************************************************************
====================
It took a while, but you finally made me happy.
*******************************************************************************************************************
To: Luna -  I don’t even
====================
You best believe I know you hate me, I'll tell you that in person when the sun comes up.
*******************************************************************************************************************
To
====================
I want to thank you for always being there for me. I've never been in such a good place in my life and I couldn't let you
====================
My name is Eli, and I love you.
*******************************************************************************************************************
To: ejuan -  you were the first person i
====================
Your opinion matters. Please don’t leave me.
*******************************************************************************************************************
To: Luke -  I wish you were more like
====================
It is a sad and sad story.
I was in love with you. I was so naive. I thought maybe you liked me too much to
====================
What a beautiful song. But I think I'm done crying yet.
*******************************************************************************************************************
To: emily -  Thank you for
====================
‹’ I love you, but i have to spend more time with you.
’’I love you, but i have
====================
To: carter -  i really liked you but you hurt me the most i thought you did. i wish we could go back to the way
====================
You can do your best to tell me you just love me, but I can always use a few more of you.
*******************************************************************************************************************
====================
The Quantum Break demo released today is already the second most downloaded game on the App Store.
*******************************************************************************************************************
To: Roshan
====================
You will love the fact that I'm not afraid to talk to you about anything. I love having you as a friend and I honestly don't know
====================
The first time I ever met her was when i was a kid. I was just looking at the sky and seeing the sunsets and I thought to
====================
You are where I thought you would be.
*******************************************************************************************************************
To: julius -  i just want to be in a
====================
I'm not sure I'll ever trust another one.
*******************************************************************************************************************
To: Nathanael -  Why can't we go
====================
‹Is he married ?’’s he ?’s she ?’s she ?’s she ?’
====================
The first time I saw your name I thought of you as a friend. I thought you were my first love.
*******************************************************************************************************************

====================
5.13 Ranked #13607 Popularity #12335 Members 5,194 Novel Yumihara, Ono (Story) Add to List Select
====================
And so, I went to the beach with you, and you ran around with her. I wanted to ask you how you felt about it, but
====================
So many people have told me I will one day 1,000,000,000. I can't believe I'm so optimistic. I don't
====================
You've probably noticed that I'm usually more of a loner, but I love you more than anything. I know you
====================
A perfect day for a funeral.

I wish I could have imagined what I would have done without you.
********************************
====================
I know my mouth is a mess, but I will still answer your emails.
*******************************************************************************************************************
To:
====================
Our customers are calling us asking us to list our stuff. We're not ready for that yet. I do want to list
====================
We were best friends as freshmen and were inseparable. I miss you so much.
*******************************************************************************************************************
To
====================
Thank you for your thoughtful comments that made me more confident in my future with my life. I know you did and will
====================
This is a rush transcript. Copy may not be in its final form.

JUAN GONZÁ
====================
Pretoria - Anwar Ali and his family are living on a farm in Kimberley. He's in his late teens
====================
›› What can you expect from me if i don’t treat you how you deserve to be treated?
====================
Lifestyle
Feet and arms? they should be yours.
- K.
*******************************************************************************************************************
To
====================
The day after I had my birthday, I woke up to a message from my best friend. We were talking about the future
====================
How can I be bad or good if I'm not in the moment?

I'm sorry for the past and I
====================
Yasmin: I love you so much. I'm so happy we can be best friends again, and I hope one
====================
I don’t think you are ready for me yet.
*******************************************************************************************************************
To: Cade -
====================
At the moment of writing this, I am still walking around empty handed. I've got enough troubles without having to deal with
====================
If I had the chance to marry my best friend, I would take it to the limit. She's the best. No
====================
I'm still in love with you. You've changed much since we last talked and it's not because you're more capable
====================
I have a feeling that my ass is on fire.
*******************************************************************************************************************
To: jake -  you
====================
We've recovered our first-floor downstairs office, but we've lost our energy. We get tripped up on our phones
====================
›››› I’m sorry for how I treated you so long ago. I just wish
====================
You know how you can't have her?

I know we could be something but i like her more than you will
====================
To: Kai -  I want you to love me more than your own skin.
*******************************************************************************************************************
To
====================
*******************************************************************************************************************
To: katie -  i miss you so much. i love you
****************************************************************
====================
It's been a while. I'm sorry I don't care anymore. I hope you don't feel the same.

====================
” I’m sorry for everything.’ I’m sorry I was too much of myself for you
====================
You have to admit, I'm impressed. I am. I took you to this school because you were in a positive relationship
====================
People need to stop calling me by my nickname or rung my doorbell.

Because you know that sounds stupid.
====================
You have no idea how much I love you. I love you. You're the best friend I've ever been able to
====================
Bishop: You are the most important person to me. 
Because of you, I feel like I am not good
====================
Liz M. (born Mar. 11, 1985) is an American actress. She is currently best known for her recurring
====================
M: I don’t know. I just want you to be happy.
*******************************************************************************************************************
To
====================
It has been two years since I met you and I still don’t know what to say to you.
********************************
====================
This article is about a/an.

See also:

Background

The other day I was playing in
====================
AEST: 23:00
SEPTEMBER: 23:00
SEPTEMBER: 23:00
SEPT
====================
I'm most proud of you. You're the bestest person I've ever met. I love you.
****************************************************************
====================
If you've ever been to a party, you know it's a tough one to leave. Some people are so good to
====================
We were not meant to be.

The 2nd grade was a rough time for us. We were struggling to read
====================
I sometimes wonder why I don't have the same feelings anymore.
*******************************************************************************************************************
To: Aiden -
====================
You didn’t do a good job as a mother, did you?
*******************************************************************************************************************
To:
====================
How did you even get to be friends with someone so oblivious to what you have to offer two years later?
****************************************************************
====================
o I don’t know what will happen to you. It's been ten years but I still miss you.
*******************************************************************************************************************
====================
We have been together for almost 6 months but I still think of you every time I hear the word "woman"
*******************************************************************************************************************

====================
From Wowpedia

The Shining's fire is so strong that it burns the entire world and leaves behind a trail of destruction through seconds.
********************************
====================
Every time I write a story, I wonder, "Does this make sense?"

Do I want to hurt the girl that I love so
====================
I'd like to thank all the people that have helped me, especially my brother, he has taught me so much about who I am and how to
====================
Lately I've realized that I am not a person. I am an object.
*******************************************************************************************************************
To: Jodie -
====================
” I thought you would care about me. I thought you would be my best friend.’
-in a thick accent
T:
====================
Songs by:
"Here Comes the Sun" - Frankie - I can feel the sun on my skin, I can feel the heat on my
====================
I'm glad I did. I'm glad I was able to look you in the eye and tell you how much I loved you.
****************************************************************
====================
A couple of days ago, I received a note from someone who claimed to be from the band from the movie Forever. I was so happy. I
====================
This post is about my childhood and the time I spent with my family. I hope you're well.
*******************************************************************************************************************
To:
====================
To: Scott -  You're the smartest person I've ever met, and I think I can live without you.
*******************************************************************************************************************
====================
This will be your last chance to be the person I loved. I hope you know that.
*******************************************************************************************************************
To: Ty -
====================
To: spencer -  i don’t think i ever will. but i do hope that in time you don’t see this
====================
Manny C.
I've loved you since we first met, but now you've made me hate you.
-Honeybear
-
====================
Kegel is a tool for a very specific reason.

If you are one of those people who doesn't like to talk to you about
====================
Look, I get it. I had nothing to do with it. I had to be there for you. I know I fucked up. But the
====================
I would love to be able to tell you how I felt, but I don't think I can.
*******************************************************************************************************************
To
====================
You are the best friend you will ever have?
-B
- I am so jealous of how you treated me. We were so good.
====================
I'm often asked what I think about my girls. I think about you so much. I miss you.
*******************************************************************************************************************
'''

outputList = output.split("====================")



positiveTexts = []
negativeTexts = []
#Here I iterate through the list, run sentiment analysis on all the generated texts, and divide them into positive and negative texts
for i in range(len(outputList)-1):
  outputList[i] = outputList[i].strip()
  outputList[i] = outputList[i].replace("\n"," ")
  outputList[i] = outputList[i].replace("*","")
  sentAnalysisText= TextBlob(outputList[i])
  polarity = sentAnalysisText.sentiment.polarity
  if polarity > 0: 
   positiveTexts.append(outputList[i])
  elif polarity < 0:
    negativeTexts.append(outputList[i])
  elif polarity == 0:
    i+=1
 


In [ ]:
#Part 4 - The Conversation Toolbox

running = True
 
#I ran this loop while conversing with the strangers. First it asks me to enter the text I received from the stranger and then choose whether or not I want to
#include a prompt. If I just include the received text, the loop runs sentiment analysis on the received text and gives me a corresponding generated text from
#the lists generated earlier. 

#However, if i add a prompt, it uses it as a "prefix" and generates a new text using GPT-2
while running:
  receivedText = input("Received Text: ")
  prompt = input("prompt? ")

  if prompt == "no" or "NO" or "No":
    receivedTextBlob = TextBlob(receivedText)
    receivedTextSentiment = receivedTextBlob.sentiment.polarity

    if receivedTextSentiment > 0:
      print(positiveTexts[random.randint(0,len(positiveTexts)-1)])

    elif receivedTextSentiment < 0:
      print(negativeTexts[random.randint(0,len(negativeTexts)-1)])

    elif receivedTextSentiment == 0:
      listPicker = random.randint(1,3)
      if listPicker == 1:
        print(positiveTexts[random.randint(0,len(positiveTexts)-1)])
      elif listPicker == 2:
        print(negativeTexts[random.randint(0,len(negativeTexts)-1)])

  else:
    gpt2.generate(sess,
                run_name=run_name,
                length=30,
                temperature=0.7,
                prefix=prompt,
                nsamples=1,
                batch_size=1
                )



  run = input("Run Again?: ")
  if run == "yes" or "YES" or "Yes":
    running = True
  elif run == "no" or "NO" or "No":
    running = False
    


Received Text: I hate you
prompt? no
Greetings, missy, are you sick or something?  To:Matt -  I miss you, though I
Run Again?: yes
Received Text: Why are you like this?
prompt? I hate you
Run Again?: yes
Received Text: I hate you
prompt? why are u like this
"I'm sorry I ruined your life. -Will" –Isaac, 17 After last night, I'm starting to think I
Run Again?: no


KeyboardInterrupt: ignored